# ***PRÁCTICO FINAL DE CIENCIA DE DATOS I - Coder House***
Estudiante: **Gonzalo Leonel Gramajo**  
Comisión: **74920**  
Documento: **40441349**  
Año: **2025**

____________________________________________________________________________________________
## **1. INTRODUCCIÓN Y OBJETIVO**

Este notebook corresponde al trabajo final del curso "Fundamentos de ciencia de datos" de Coder House. Está basado en un dataset de ventas de autos rescatado desde Keegle. Este conjunto de datos incluye todas las entradas de vehículos usados ​​dentro de los Estados Unidos en Craigslist.com.
Es importante resaltar que se cuenta con 426 mil lineas y el archivo ocupa 1.45 GB de almacenamiento, por lo que un archivo de tal tamaño no se peude subir a GitHub. Dado esto, el archivo .csv se puede encontrar disponible en Google Drive y obviamente, Kaggle.

### **Dataset**
Used Cars Dataset - Vehicles listings from Craigslist.org.  
Enlace web a Kaggle: https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data  
Enlace web a Google Drive: https://drive.google.com/file/d/1uQ_YhqBimI46j5W-EgSwkjZvpFt87ejt/view?usp=sharing

### **Objetivo**
*Mediante el uso de algortimos de aprendizaje automático, lograr predecir el precio de venta de los vehículos según los datos de entrada odómetro (kilometraje), year (año de fabricación), transmission (trasmisión), fuel (combustible).*

**Pasos para lograr el objetivo:**
1.  **Introducción y Objetivo:** Definir el problema y el objetivo.
2.  **Importar Librerías:** Cargar las herramientas necesarias.
3.  **Carga de Datos:** Leer el dataset.
4.  **Hipótesis:** Plantear una hipótesis clara.
5.  **Análisis Exploratorio de Datos (EDA):** Entender los datos, distribuciones, valores faltantes y relaciones.
6.  **Feature Engineering (Ingeniería de Características):** Crear nuevas características si es relevante.
7.  **Preprocesamiento de Datos:** Preparar los datos para el modelo (manejo de categorías, escalado).
8.  **División de Datos:** Separar en conjuntos de entrenamiento y prueba.
9.  **Construcción y Entrenamiento del Modelo:** Usar RandomForestClassifier.
10. **Evaluación del Modelo:** Medir el rendimiento con métricas adecuadas.
11. **Análisis de Importancia de Características:** Identificar qué variables son más influyentes.
12. **Conclusiones:** Interpretar los resultados y validar/refutar la hipótesis.

___________________________________________________________________________________________
## **2. IMPORTACIÓN DE MÓDULOS**

Para cargar todos los módulos (herramientas) que se necesitan en el despliegue de todo el análisis y entrenamiento, se debe ejecutar el siguiente código:

In [ ]:
# Herramientas principales
import pandas as pd
import numpy as np
# Módulos de visualización
import matplotlib.pyplot as plt
import seaborn as sns
# Módulo de preprocesamiento
from sklearn.preprocessing import LabelEncoder
# Módulo de división de datos
from sklearn.model_selection import train_test_split
# Módulo de regresión
from sklearn.ensemble import RandomForestRegressor
# Módulo de evaluación
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

___________________________________________________________________________________________
## **3. CARGA DE DATOS**

Para conseguir un dataframe de los datos que contiene el .csv del dataset rescatado, se debe ejecutar el siguiente código:

In [ ]:
file_path = './vehicles.csv' # ruta al archivo CSV
df = pd.read_csv(file_path) # leer el archivo CSV y conseguir un dataframe

In [ ]:
df.head(10) # mostrar los primeros 5 valores

In [ ]:
df.tail(10) # mostrar los últimos 5 valores

_____________________________________________________________________________
## **4. HIPÓTESIS**

Las siguientes hipótesis se plantean en marco al objetivo general de este análisis:  

1. **Los vehículos con valores en el odómetro más bajo se venden exponencialmente más caros.**  
*Esto es porque los vehículos con menos kilometraje (en teoría) deberían estar mas nuevos y en mejor estado que los autos que recorrieron más kilometros. En base a esto, se hipotetiza que la variación de este precio es de tipo exponencial.*  
  
2. **Los vehículos con año de fabricación menor, se venden exponencialmente más caros.**  
*Siguiendo con la premisa de que los autos más nuevos valen exponencialmente menos, se tratará de comprobar si esto también va en función del año de fabricación del vehículo y que correlación existe con la hipótesis anterior.*  

3. **Más del 50% de las publicaciones, son de vehículos con transmisión automática.**  
*Teniendo en cuenta la premisa que los automóviles con trasmisión automática son los más valuados, se verificará si esto es efectivamente así y que relación tiene con las 2 hipótesis anteriores.*

______________________________________________________________________________
## **5. ANÁLISIS EXPLORATORIO DE LOS DATOS (EDA)**

1. Verificar la integridad de los datos en las columnas. Esto se hace para verificar que tan buena es la calidad del dataset. Keggle, la pagina web desde donde se descargo el datase, muestra un reporte de la integridad de los datos de cada columna, pero para complementar el análisis, se realiza el siguiente codigo:

In [ ]:
# Función para calcular el porcentaje de integridad de cada columna
def column_integrity_simple(df):
    for col in df.columns:
        total_rows = len(df)
        non_null_count = df[col].notna().sum()
        integrity_percentage = (non_null_count / total_rows) * 100
        print(f"Columna {col}: {round(integrity_percentage, 2)}% -> {total_rows - non_null_count}/{total_rows}")

# Ejecutar el análisis de integridad simple
print("Integridad de los datos por columna:")
column_integrity_simple(df)

2. Debido a que se muestra que no todas las columnas están completas, con el 100% de los datos, es interesante saber que cantidad de datos falta. Para esto se ejecuta la siguiente línea de código.

In [ ]:
df.isna().sum()

3. Si se quiere observar lo contrario, es decir, que datos son no nulos, además de que tipo de datos son y el uso de memoria del dataframe, conviene ejecutar la siguiente linea de código.

In [ ]:
df.info()

4. Por ultimo, para tener un pantallazo de alguno valores impotantes como la tendecnia central, dispersión y forma de la distribución de los datos, exluyendo los valores NaN, se debe ejecutar la siguiente linea de código.

In [ ]:
df.describe()

### Descripción de los campos

| # | CAMPO | TIPO DE DATO | DESCRIPCIÓN |
|---|-------|--------------|-------------|
| 1 | id | number | identificador del automovil publicado en la pagina web. |
| 2 | url | string | direccion web a la publicaciion del automovil en la pagina web. |
| 3 | region | string | estado de los estado unidos donde esta publicado el automovil segun Craiglist. |
| 4 | region_url | string | url pertencceinte exclusivamente a la region. |
| 5 | price | number | precio del automovil en unidades de la moneda dolar (USD). |
| 6 | year | number | año de fabricacion del automovil. |
| 7 | manufacturer | string |fabricante del vehiculo. |
| 8 | model | string | modelo del vehiculo segun la disignacion que le dio el fabricante. |
| 9 | condicion | string | esta es una evaluacion que se hace al publicar el automovil en la pagina web, pero es un dato subjetivo acorde al publicador. |
| 10 | cylinders | string | cantidad de cilindros que tiene el motor en su estructura y como estan configurados. |
| 11 | fuel | string | combustible que utiliza el vehiculo. |
| 12 | odometer | number | es la distancia recorrida todal del vehiculo en millas. |
| 13 | title_status | string | estado del titulo del vehiculo. No se conoce especificamente que significa, se averiguara mas al respecto. |
| 14 | transmission | string | tipo de trasmision con la que esta configurado el vehiculo. |
| 15 | VIN | string | numero de identificacion del vehiculo. Es parecido a la patente. |
| 16 | drive | string | tipo de traccion del vehiculo. Esto es, como se distribuye la energia mecanica a las ruedas. |
| 17 | size | number | se invertigara mas al respecto de que significa esta columna. |
| 18 | type | string | tipo o categoria del formato del vehiculo. |
| 19 | paint_color | string | color exterior del vehiculo. |
| 20 | image_url | string | link a la imagen del vehiculo publicado. |
| 21 | description | string | descripcion del vehiculo redactada por la persona que publico el vehiculo en la pagina web. |
| 22 | country | null | columna vacia ya que el valor siempre era estados unidos. |
| 23 | state | string | estado de los estados unidos donde se publica el vehiculo. No se almacena el nombre completo, sino la abreviatura. |
| 24 | lat | number | latitud de la ubicacion del vehiculo listado. |
| 25 | lon | number | longitud de la ubicacion del vehiculo listado. |
| 26 | posting_date | datetime | fecha de la publicacion del vehiculo. |

Debido a que existen datos que no se encuentran en el dataframe, se procede a completar los mismos con datos acordes al tipo que corresponde. Esto se hace con el fin de evitar errores en los proximos pasos (graficos, modelado). También es importante porque se puede completar una linea en vez de eliminarla totalmente y perder todos esos datos.  

Por otro lado, el dataframe tiene datos que no se usarán y es conveniente elimarlos para ahorrar espacio en memoria y agilizar el procesamiento. Si no fuesen muchos datos, esto tal vez no es necesario de realizar, a pesar de que se pierde prolijidad. Pero como estamos hablando de mas de un dataframe de más de 1 GB, se debe optimizar esto.

In [ ]:
# Completar datos faltantes en las columnas de tipo object/string
columnas_editar = ("manufacturer", "model", "condition",
                   "cylinders", "fuel", "title_status",
                   "transmission", "drive", "size", "type",
                   "paint_color")
for columna in columnas_editar:
    df[columna] = df[columna].fillna("uknown")

# Completar datos faltantes en la columna descripción con strings vacíos
df["description"] = df["description"].fillna("")

# Completamos los datos faltantes en las columnas de tipo number
df["year"] = df["year"].fillna(df["year"].mean())
df["odometer"] = df["odometer"].fillna(df["odometer"].mean())

# Eliminamos columnas innecesarias
columnas_eliminar = ["url","region", "VIN", "county", "lat", "long", "image_url", "region_url", "posting_date"]
df.drop(columns=columnas_eliminar, axis=1, inplace=True)

# Mostrar cuántos datos faltantes quedan
print(df.isna().sum())

Hay columnas que no fueron completadas porque no es necesario hacerlo por el tipo de dato que representan. Por ejemplo, VIN es la patente o un identificador unico del vehoiculo. Sin embargo, no son datos que sean relevantes para la contrastación de la hipotesis y no serán usados en el análisis o entrenamiento. Como así tambien la ubicacion precisa de donde se realizo el posteo, es decir "lat" y "lon".  
Luego de estos cambios, el dataframe queda con la siguiente descripción:

In [ ]:
df.info()

______________________________________________________________________________
## **5.1. CONTRASTACIÓN DE LAS HIPÓTESIS**

#### Hipótesis 1:
*''Los vehículos con valores en el odómetro más bajo se venden exponencialmente más caros.''*

Primero, se puede visualizar los datos de precio contra el odometro para poder visualizar si a simple vista, la relacion es exponencial como se plantea en la hipótesis.

In [ ]:
# Se realiza el siguiente filtrado para poder eliminar un poco los outliers y los datos que no tienen sentido.
df_filtrado = df[
    (df["odometer"] < df["odometer"].quantile(0.98)) &
    (df["price"] < df["price"].quantile(0.98)) &
    (df["price"] > 1000)
]

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_filtrado, x="odometer", y="price", alpha=0.3, s=10)
plt.title("Relación entre odómetro y precio del vehículo")
plt.xlabel("Odómetro (millas)")
plt.ylabel("Precio (USD)")
plt.grid(True)
plt.tight_layout()
plt.show()

Por la cantidad inmensa de puntos (426.000 datos), parece una nebulosa y no se percibe claramente si la relación es exponencial, lineal o de otro tipo. Se puede afirmar que hay mucha dispersión de los datos.  
Desde ya, como **conclusión**, se puede afirmar que el odómetro solo NO explica el precio de forma clara ni de forma exponencial. Puede haber una tendencia débil, pero no es dominante.  
Sí se podría filtrar los datos por marca, año o modelo para mostrar una tendecnia menos general. Para observar esto, se debe ejecutar el siguiente codigo:

In [ ]:
# Contar las marcas
marca_mas_publicada = df_filtrado["manufacturer"].value_counts().idxmax()
print(f"La marca con más publicaciones es: {marca_mas_publicada}")

# Filtrar solo esa marca
df_marca = df_filtrado[df_filtrado["manufacturer"] == marca_mas_publicada]

# Contar los modelos
modelo_mas_publicado = df_marca["model"].value_counts().idxmax()
print(f"Modelo de la marca con más publicaciones: {modelo_mas_publicado}")

# Filtrar solo ese modelo
df_marca_modelo = df_marca[df_marca["model"] == modelo_mas_publicado]

# Mostrar la relación entre el precio y el odómetro para esa marca
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_marca_modelo, x="odometer", y="price", alpha=0.3, s=10)
plt.title(f"Relación odómetro vs precio - {marca_mas_publicada} {modelo_mas_publicado}")
plt.xlabel("Odómetro (millas)")
plt.ylabel("Precio (USD)")
plt.grid(True)
plt.tight_layout()
plt.show()

Ahora si se puede ver una clara tendencia aunque hay una gran cantdad de outliers que podrían afectar el posterior modelado. La dispersión que se vé muestra mas bien una distribución lineal con pendiente negativa. Por lo tanto la hipóteisis queda refutada.

#### Hipótesis 2:
*''Los vehículos con año de fabricación menor, se venden exponencialmente más caros.''*

Primero, se puede visualizar los datos de precio contra los años para poder visualizar si a simple vista, la relacion es exponencial como se plantea en la hipótesis.

In [ ]:
# Se realiza el siguiente filtrado para poder eliminar un poco los outliers y los datos que no tienen sentido.
df_filtrado = df[
    (df["year"] < df["year"].quantile(0.99)) &
    (df["price"] < df["price"].quantile(0.99)) &
    (df["price"] > 1000)
]

In [ ]:
# Contar las marcas
marca_mas_publicada = df_filtrado["manufacturer"].value_counts().idxmax()
print(f"La marca con más publicaciones es: {marca_mas_publicada}")

# Filtrar solo esa marca
df_marca = df_filtrado[df_filtrado["manufacturer"] == marca_mas_publicada]

# Contar los modelos
modelo_mas_publicado = df_marca["model"].value_counts().idxmax()
print(f"Modelo de la marca con más publicaciones: {modelo_mas_publicado}")

# Filtrar solo ese modelo
df_marca_modelo = df_marca[df_marca["model"] == modelo_mas_publicado]

# Mostrar la relación entre el precio y el odómetro para esa marca
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_marca_modelo, x="year", y="price", alpha=0.3, s=10)
plt.title(f"Relación año vs precio - {marca_mas_publicada} {modelo_mas_publicado}")
plt.xlabel("Año")
plt.ylabel("Precio (USD)")
plt.grid(True)
plt.tight_layout()
plt.show()

En este caso se nota que si se cumple lo que se plantea la hipótesis.

#### Hipótesis 3:
*''Más del 50% de las publicaciones, son de vehículos con transmisión automática.''*

Se puede visualizar los datos en gráfico de torta para saber facilmente si esto es cierto.

In [ ]:
# Contar los valores de la columna "transmission"
conteo_transmision = df_filtrado["transmission"].value_counts()

# Mostrar los conteos (opcional, para ver qué hay)
print(conteo_transmision)

# Gráfico de torta
plt.figure(figsize=(8, 8))
plt.pie(
    conteo_transmision,
    labels=conteo_transmision.index,
    autopct='%1.1f%%',  # Muestra el porcentaje
    startangle=180,       # Para que empiece arriba
    shadow=False
)
plt.title("Distribución de tipos de transmisión")
plt.axis('equal')  # Hace que el círculo sea redondo
plt.show()

# Además, calculamos el porcentaje de "automatic"
total = conteo_transmision.sum()
automaticos = conteo_transmision.get("automatic", 0)

porcentaje_automaticos = (automaticos / total) * 100
print(f"Porcentaje de vehículos automáticos: {porcentaje_automaticos:.2f}%")


En esta ocasión, tambien queda demostrada la hipótesis, ya que el 78,88% de los vehículos son con transmisión automática.

______________________________________________________________________________
## **6. FEATURE ENGINEERING (Ingeniería de características)**

Como se pudo ver en las hipótesis, los datos tienen una gran dispersión y muchos outliers. Esto es perjudicial para el modelo, por lo que se filtrarán los datos outliers para lograr un modelo más fiable. Para eso se debe ejecutar el siguiente codigo.

In [ ]:
df = df[
    (df["odometer"] < df["odometer"].quantile(0.98)) &
    (df["year"] >= 1960) &
    (df["year"] <= 2024) &
    (df["price"] > 1000) &
    (df["price"] < df["price"].quantile(0.98))
]

Se crearán las siguientes nuevas caracterísitcas para poder lograr un entrenamiento factible que logre el objetivo:  

1. **Antigüedad del vehículo:** esta característica logra un valor que es más intuitivo que el año. A mayor edad, menor precio.
2. **Kilometraje promedio por año:** esta nueva característica da idea del uso intensivo. Un auto más antigüo pero poco usado puede valer más que uno nuevo con mucho kilometraje.
3. **Densidad de publicaciones por estado:** esto mide cuántos autos hay por estado. En base a esto se puede deducir como afecta la ley de oferta/demanda.
4. **Longitud del texto de descripción:** En ventas, una descripción larga a veces significa más detalle, más confianza del vendedor, por lo tanto esto podría afectar al precio de venta.

In [ ]:
# Antigüedad del vehículo
df["vehicle_age"] = 2025 - df["year"]
# Kilometraje promedio por año
df["miles_per_year"] = df["odometer"] / df["vehicle_age"]
# Densidad de publicaciones por estado
state_counts = df["state"].value_counts().to_dict()
df["state_density"] = df["state"].map(state_counts)
# Longitud del texto de descripción
df["desc_length"] = df["description"].apply(lambda x: len(str(x)))

Una vez ejecutado este código, el dataframe queda como sigue:

In [ ]:
df.info()
df.head()   

______________________________________________________________________________
## **7. PREPROCESAMIENTO DE DATOS**

Con respecto a este paso, en primer lugar se aborda el manejo de las variables categóricas. Esto se hace porque lo que se busca es predecir el precio de un vehículo. Por lo tanto, se estaría hablando de problema de regresión ya que, el precio, es un valor numérico contínuo.  

Cuando el problema es de este tipo, las variables categoricas que tiene el modelo se deben pasar a variables numéricas. Para esto se utilizará la técnica Label Encoding, que consiste en asignar un numero a cada categoría de los campos categóricos presentes en el dataset y que son importantes para el entrenamiento del modelo predictivo.  

Es importante aclarar también que, dada la cantidad de variables a disposición, afectarían al precio que se predice. Por lo tanto, se hace uso de un algoritmo del tipo "Árbol de regresión", más precisamente el algoritmo **RandomForestRegressor**. El "Forest" del nombre quiere decir que Random Forest combina muchos árboles diferentes y toma el promedio de sus predicciones, lo que reduce el sobreajuste y mejora la precisión respecto a usar un solo árbol.

In [ ]:
# Iniciar instancia del codificador
label_encoder = LabelEncoder()

# Codificación de las columnas categóricas
df['transmission_encoded'] = label_encoder.fit_transform(df['transmission'])
df['manufacturer_encoded'] = label_encoder.fit_transform(df['manufacturer'])
df['model_encoded'] = label_encoder.fit_transform(df['model'])
df['type_encoded'] = label_encoder.fit_transform(df['type'])
df['fuel_encoded'] = label_encoder.fit_transform(df['fuel'])

# Mostrar información del DataFrame después de la codificación
print("\nInformación del DataFrame después de la codificación:")
df.info()

______________________________________________________________________________
## **8. DIVISIÓN DE DATOS**

Se procede a dividir el conjunto de datos en los correspondientes al entrenamiento y los de test.

In [ ]:
# Definición de variables predictoras (X) y variable objetivo (y)

X = df[[ # Variables predictoras
    'transmission_encoded',
    'manufacturer_encoded',
    'model_encoded',
    'type_encoded',
    'fuel_encoded',
    'miles_per_year',
    'vehicle_age',
    'state_density',
    'desc_length']]

y = df['price']  # Columna que se va a predecir

# Dividimos en 80% entrenamiento y 20% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

______________________________________________________________________________
## **9. CONSTRUCCIÓN Y ENTRENAMIENTO DEL MODELO**

In [ ]:
# Instanciar el modelo
# n_estimators: número de árboles en el bosque
# random_state: semilla para la aleatoriedad (reproducibilidad)
modelo = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenar el modelo con los datos de entrenamiento
modelo.fit(X_train, y_train)

______________________________________________________________________________
## **10. EVALUACIÓN DEL MODELO**

Se usan las 3 métricas más comunes para modelos de regresión, es decir:

- **MAE (Mean Absolute Error)**: cuánto se equivoca el modelo en promedio, sin importar el signo.
- **RMSE (Root Mean Squared Error)**: penaliza más los errores grandes.
- **R² (coeficiente de determinación)**: qué proporción de la variabilidad del precio explica el modelo (1 = perfecto, 0 = no explica nada).

In [ ]:
# Hacemos predicciones sobre el conjunto de prueba
y_pred = modelo.predict(X_test)

# Calculamos las métricas
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Mostramos los resultados
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.2f}")

Estas metricas nos quiere decir que:
1. El modelo se equivoca en promedio por $1.755, un error muy bajo y aceptable.
2. Los errores grandes son penalizados más; aun así, sigue siendo bajo.
3. El modelo explica el 91% de la variabilidad del precio.

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')
plt.xlabel('Precio real')
plt.ylabel('Precio predicho')
plt.title('Comparación: Precio real vs. Precio predicho')
plt.grid(True)
plt.tight_layout()
plt.show()


______________________________________________________________________________
## **11. ANÁLISIS DE IMPORTANCIA DE CARACTERÍSTICAS**

La importancia de características indica cuánto contribuye cada variable a las predicciones del modelo.
En RandomForestRegressor, esto se calcula automáticamente en base a cuánta “información” aporta cada variable al reducir el error.  
Para comprobar que variables son las más relevantes, se debe ejecutar el siguiente código:

In [ ]:
# Obtener la importancia de cada feature
importancias = modelo.feature_importances_
nombres_features = X.columns

# Crear un DataFrame ordenado
df_importancias = pd.DataFrame({
    'Feature': nombres_features,
    'Importancia': importancias
}).sort_values(by='Importancia', ascending=True)

# Gráfico de barras horizontal
plt.figure(figsize=(10, 6))
plt.barh(df_importancias['Feature'], df_importancias['Importancia'], color='skyblue')
plt.xlabel('Importancia')
plt.title('Importancia de cada Característica en el Modelo')
plt.tight_layout()
plt.show()


______________________________________________________________________________
## **12. CONCLUSIONES**

Se concluye en que el modelo obtenido cumple con el objetivo de brindar un precio estimado para un vehículo acorde al año, kilometraje, fabricante, modelo, kilometros por año, tipo de combustible, typo de carrocería, densidad de publicaciones por estado y tipo de transmisión.  

También se concluye en que el modelo tiene unos parámetros de desempeño aceptables, pero mejorables. La fuente de datos no es de lo más fiable (Criglist) ya que existen muchos valores que no son obligatorios al momento de realizar una publicación.

A modo de comclusión académica o de aprendizaje, cabe mencionar lo sustancialmente divertido que fue lograr un modelo que prediga el precio de un vehículo. Pude fusionar dos pasiones, la ciencia de datos con el mundo automotor.  

______________________________________________________________________________
## **13. PRÓXIMAS LÍNEAS**

1. Para mejorar el modelo se puede probar con otros algoritmos como ser XGBoost, que es un algortmo de Gradient Boosting o "aumento de gradiente". Teóricamente daría mejores resultados que RandomForest.  

2. Para simplificar el modelo, se puede considerar eliminar las features transmission_encoded y state_density ya que no tienen una gran importancia en la minimización del error.  

3. Investigar que otras caracterisitcas se podrían agregar para que el modelo sea mas preciso y completo.  

4. Generar un codigo en python para ingresar, codificar y decodificar los valores de un nuevo vehículo pasado al modelo para que realice la detección de su precio.